## Load Libraries & Datasets

In [1]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression
from scipy.io import savemat

from sklearn.model_selection import train_test_split
from sklearn.svm import SVC, LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.ensemble import RandomForestClassifier 
from sklearn.model_selection import cross_val_score, cross_validate
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import GaussianNB
from keras.utils.np_utils import to_categorical

from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report, accuracy_score, matthews_corrcoef, balanced_accuracy_score, precision_recall_fscore_support
from sklearn.metrics import auc, average_precision_score, precision_recall_curve, roc_curve

In [2]:
from keras.models import Model
from keras.layers import Input, Dense, BatchNormalization, Dropout, Conv1D, Conv2D, MaxPooling2D, MaxPooling1D, Flatten
from keras import optimizers
from keras import metrics

from keras import backend as K
from keras.models import load_model
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from random import sample

In [3]:
def prepare_dset(dset):
    dset = pd.concat([pd.DataFrame(np.asarray(dset.conc > 0.01, dtype='int')).rename(columns={0:'label'}), 
           dset.iloc[:, 3:]], axis=1)
    return dset

In [4]:
## raw data
r6g_b1_raw = pd.read_csv("./data_final/r6g_dset/r6g_batch1.csv")
r6g_b2_raw = pd.read_csv("./data_final/r6g_dset/r6g_batch2.csv")
r6g_b1_raw = prepare_dset(r6g_b1_raw)
r6g_b2_raw = prepare_dset(r6g_b2_raw)

## Baseline Corrected
r6g_b1_bl = pd.read_csv("./data_final/r6g_dset/r6g_bl_batch1.csv")
r6g_b2_bl = pd.read_csv("./data_final/r6g_dset/r6g_bl_batch2.csv")
r6g_b1_bl = prepare_dset(r6g_b1_bl)
r6g_b2_bl = prepare_dset(r6g_b2_bl)

## Power Spectrum Normalized
r6g_b1_psn = pd.read_csv("./data_final/r6g_dset/r6g_psn_batch1.csv")
r6g_b2_psn = pd.read_csv("./data_final/r6g_dset/r6g_psn_batch2.csv")
r6g_b1_psn = prepare_dset(r6g_b1_psn)
r6g_b2_psn = prepare_dset(r6g_b2_psn)

## Batch Normalized
r6g_b1_bn = pd.read_csv("./data_final/r6g_dset/r6g_bn_batch1.csv")
r6g_b2_bn = pd.read_csv("./data_final/r6g_dset/r6g_bn_batch2.csv")
r6g_b1_bn = prepare_dset(r6g_b1_bn)
r6g_b2_bn = prepare_dset(r6g_b2_bn)

## Baseline Correctd & Power Spectrum Normalized
r6g_b1_bl_psn = pd.read_csv("./data_final/r6g_dset/r6g_bl_psn_batch1.csv")
r6g_b2_bl_psn = pd.read_csv("./data_final/r6g_dset/r6g_bl_psn_batch2.csv")
r6g_b1_bl_psn = prepare_dset(r6g_b1_bl_psn)
r6g_b2_bl_psn = prepare_dset(r6g_b2_bl_psn)

## Baseline Correctd & Batch Normalized
r6g_b1_bl_bn = pd.read_csv("./data_final/r6g_dset/r6g_bl_bn_batch1.csv")
r6g_b2_bl_bn = pd.read_csv("./data_final/r6g_dset/r6g_bl_bn_batch2.csv")
r6g_b1_bl_bn = prepare_dset(r6g_b1_bl_bn)
r6g_b2_bl_bn = prepare_dset(r6g_b2_bl_bn)

## Define Util Functions

In [5]:
## Define performance measures
def yoden_index(y, y_pred):
    tn, fp, fn, tp = confusion_matrix(y, y_pred, labels=[0,1]).ravel()
    j = (tp/(tp+fn)) + (tn/(tn+fp)) - 1
    return j

def pmeasure(y, y_pred):
    tn, fp, fn, tp = confusion_matrix(y, y_pred, labels=[0,1]).ravel()
    sensitivity = tp / (tp + fn )
    specificity = tn / (tn + fp)
    f1score = (2 * tp) / (2 * tp + fp + fn)
    return ({'Sensitivity': sensitivity, 'Specificity': specificity, 'F1-Score': f1score})

def Show_Statistics(msg,mean_Stats, sd_Stats, sigfig):
    print(msg.upper())
    print(70*'-')
    print('Accuracy:{} + {}'          .format(round(mean_Stats[0],sigfig), round(sd_Stats[0],sigfig)))
    print('Sensitivity:{} + {} '      .format(round(mean_Stats[1],sigfig), round(sd_Stats[1],sigfig)))
    print('Specificity:{} + {}'       .format(round(mean_Stats[2],sigfig), round(sd_Stats[2],sigfig)))
    print('F1-Score:{} + {}'          .format(round(mean_Stats[3],sigfig), round(sd_Stats[3],sigfig)))
    print('MCC:{} + {}'               .format(round(mean_Stats[4],sigfig), round(sd_Stats[4],sigfig)))
    print('Balance Accuracy:{} + {}'  .format(round(mean_Stats[5],sigfig), round(sd_Stats[5],sigfig)))
    print('Youden-Index:{} + {}'      .format(round(mean_Stats[6],sigfig), round(sd_Stats[6],sigfig)))
    print('AUC:{} + {}'               .format(round(mean_Stats[7],sigfig), round(sd_Stats[7],sigfig)))
    print('AUPR:{} + {}'              .format(round(mean_Stats[8],sigfig), round(sd_Stats[8],sigfig)))  
    print(70*'-')

def Calculate_Stats(y_actual,y_pred, y_score):
    acc = accuracy_score(y_actual, y_pred)
    sen = pmeasure(y_actual, y_pred)['Sensitivity']
    spe = pmeasure(y_actual, y_pred)['Specificity']
    f1 = pmeasure(y_actual, y_pred)['F1-Score']
    mcc = matthews_corrcoef(y_actual, y_pred)
    bacc = balanced_accuracy_score(y_actual, y_pred)
    yi = yoden_index(y_actual, y_pred)
    #auc = roc_auc_score(y_actual.argmax(axis=1), y_pred.argmax(axis=1))
  
    pre, rec, _ = precision_recall_curve(y_actual, y_score, pos_label=1)
    fpr, tpr, _ = roc_curve(y_actual, y_score, pos_label=1)
    auroc = auc(fpr, tpr)
    aupr = auc(rec, pre)
    
    return acc, sen, spe, f1, mcc, bacc, yi, auroc, aupr  

def label_by_th(y_pred, threshold=0.5):
    y_pred_copy = y_pred.copy()
    y_pred_copy[y_pred>= threshold] = 1 
    y_pred_copy[y_pred<threshold] = 0 
    return y_pred_copy

def cutoff_youdens_j(fpr,tpr,thresholds):
    j_scores = tpr-fpr
    j_ordered = sorted(zip(j_scores,thresholds))
    return j_ordered[-1][1]

In [6]:
def get_perf_summary(dataset, model, type):
    ## Prepare dataset
    dataset.loc[dataset.label>=1,"label"]=1
    
    ## Datasets 
    [X, y_true] = np.array(dataset.iloc[:,1:]), np.array(dataset.iloc[:,0])
    y_true = to_categorical(y_true)
    if type=="regression":
        y_score = model.predict(X, batch_size=1800, verbose=0)    
    else:
        y_score = model.predict(X, batch_size=1800, verbose=0)[:,1]
    #y_score = model.predict(X,batch_size=1800, verbose=0)
    #y_pred = to_categorical(get_label(y_score))      

    # Optimal Threshold
    fpr, tpr, thresholds_AUC = roc_curve(y_true.argmax(axis=1), y_score)
    precision, recall, thresholds_AUPR = precision_recall_curve(y_true.argmax(axis=1),y_score)

    ## Optimal Threshold metrics
    distance = (1-fpr)**2+(1-tpr)**2
    EERs = (1-recall)/(1-precision)
    positive = sum(y_true.argmax(axis=1))
    negative = y_true.shape[0]-positive
    ratio = negative/positive
    opt_t_AUC = thresholds_AUC[np.argmin(distance)]
    opt_t_AUPR = thresholds_AUPR[np.argmin(np.abs(EERs-ratio))]
    opt_yodens_j = cutoff_youdens_j(fpr, tpr, thresholds_AUC)        
    y_pred_th = label_by_th(y_score, opt_yodens_j)
    y_pred = to_categorical(y_pred_th)

    print(confusion_matrix(y_true.argmax(axis=1), y_pred.argmax(axis=1), labels=[0,1]).ravel())    

    ## Measures
    t_acc, t_sen, t_spe, t_f1, t_mcc, t_bacc, t_yi, t_auc, t_aupr = Calculate_Stats(y_true, y_pred, y_score)

    print("ACC: {}, Sen: {}, Spe: {}, F1: {}, MCC: {}, BACC: {}, YI: {}, AUC: {}, AUPR: {}".format(t_acc, t_sen, t_spe, t_f1, t_mcc, t_bacc, t_yi, t_auc, t_aupr))

In [7]:
def get_scale_dset(dset):
    res =[]
    tmp_dset = dset.iloc[:,1:]
    for i, row in tmp_dset.iterrows():
        res.append(row/max(abs(row)))
    
    res = pd.DataFrame(res).reset_index(drop=True)
    res = pd.concat([dset.iloc[:,0], res], axis=1)
    
    return res 

## Define Classifiers

In [8]:
def get_clf_res_ML(dev_dset, ind_test_dset, method):
    num_Trials = 10    
    dev_dset = get_scale_dset(dev_dset)
    ind_test_dset = get_scale_dset(ind_test_dset)

    # import warnings filter
    from warnings import simplefilter
    # ignore all future warnings
    simplefilter(action='ignore', category=FutureWarning)
    [DataX, LabelY] = np.array(dev_dset.iloc[:,1:]), np.array(dev_dset.iloc[:,0])    

    Stats =[]
    Stats_Ind =[]
    for j in range(0, num_Trials):        
        X_train, X_val, y_train, y_val =\
        train_test_split(DataX, LabelY, test_size=0.2, stratify = LabelY)
        X_val, X_test, y_val, y_test =\
        train_test_split(X_val, y_val, test_size=0.5, stratify = y_val)

        if method == "LR":
            clf = LogisticRegression(random_state=0).fit(X_train, y_train)

        elif method == "LinSVM":
            clf = SVC(kernel = 'linear', probability=True).fit(X_train, y_train)

        elif method == "NB":
            clf = GaussianNB().fit(X_train, y_train)

        elif method == "DT":
            clf = DecisionTreeClassifier().fit(X_train, y_train)

        elif method == "MLP":
            clf = MLPClassifier().fit(X_train, y_train)

        elif method == "RF":
            clf = RandomForestClassifier().fit(X_train, y_train)

        elif method == "RBFSVM":
            clf = SVC(kernel = 'rbf', probability=True).fit(X_train, y_train)

        else:
            print("Please check avilable models")
            break;

        y_train_pred = clf.predict(X_train)
        y_train_score = clf.predict_proba(X_train)     
        y_train_pred = to_categorical(y_train_score.argmax(axis=1))
        y_train_score = y_train_score[:,1]

        y_val_pred = clf.predict(X_val)
        y_val_score = clf.predict_proba(X_val)     
        y_val_pred = to_categorical(y_val_score.argmax(axis=1))
        y_val_score = y_val_score[:,1]

        y_test_pred = clf.predict(X_test)
        y_test_score = clf.predict_proba(X_test) 
        y_test_pred = to_categorical(y_test_score.argmax(axis=1))
        y_test_score = y_test_score[:,1] 

        print(confusion_matrix(y_test, y_test_pred.argmax(axis=1), labels=[0,1]).ravel())

        y_train_pred = y_train_pred.argmax(axis=1)
        y_val_pred = y_val_pred.argmax(axis=1)
        y_test_pred = y_test_pred.argmax(axis=1)

        ## Training Measures
        tr_acc, tr_sen, tr_spe, tr_f1, tr_mcc, tr_bacc, tr_yi, tr_auc, tr_aupr = Calculate_Stats(y_train,y_train_pred, y_train_score);

        ## Validation Measures
        v_acc, v_sen, v_spe, v_f1, v_mcc, v_bacc, v_yi, v_auc, v_aupr = Calculate_Stats(y_val,y_val_pred, y_val_score);

        ## Test Measures
        t_acc, t_sen, t_spe, t_f1, t_mcc, t_bacc, t_yi, t_auc, t_aupr = Calculate_Stats(y_test,y_test_pred, y_test_score);

        Stats.append([tr_acc, tr_sen, tr_spe, tr_f1, tr_mcc, tr_bacc, tr_yi, tr_auc, tr_aupr,
                      v_acc, v_sen, v_spe, v_f1, v_mcc, v_bacc, v_yi, v_auc, v_aupr,
                      t_acc, t_sen, t_spe, t_f1, t_mcc, t_bacc, t_yi, t_auc, t_aupr])  

        print(' \nTraining/ Validation / Test BACC :', tr_bacc,'/',v_bacc,'/',t_bacc,
              ' \nTraining/ Validation / Test Youden-index:', tr_yi,'/',v_yi,'/',t_yi,
              ' \nTraining/ Validation / Test MCC:', tr_mcc,'/',v_mcc,'/',t_mcc,
              ' \nTraining/ Validation / Test AUC:', tr_auc,'/',v_auc,'/',t_auc,
              ' \nTraining/ Validation / Test AUPR:', tr_aupr,'/',v_aupr,'/',t_aupr)    

        ## Independent Test
        [Xt1, Yt1] = np.array(ind_test_dset.iloc[:,1:]), np.array(ind_test_dset.iloc[:,0])

        Yt1_pred = clf.predict(Xt1)
        Yt1_score = clf.predict_proba(Xt1)

        Stats_Ind.append(Calculate_Stats(Yt1, Yt1_pred, Yt1_score[:,1]))

        print(Stats_Ind)
    
    return Stats, Stats_Ind

In [9]:
def format_perf_mat(perf_list, model_type):
    perf_tbl = pd.DataFrame(perf_list)
    perf_tbl = pd.concat([perf_tbl, pd.DataFrame(np.repeat(model_type, len(perf_list)))], axis=1)
    perf_tbl.columns = ['Accuracy', "Sensitivity", "Specificity", "F1", "MCC", "BACC", 
                        "Yoden Index", "AUROC", "AUPR", "Model"]    
    return perf_tbl

In [10]:
def get_all_clf_res(devset, indset, species, prep_method):
    NB_cv, NB_ind = get_clf_res_ML(devset, indset, method='NB')
    DT_cv, DT_ind = get_clf_res_ML(devset, indset, method='DT')
    LR_cv, LR_ind = get_clf_res_ML(devset, indset, method='LR')
    RF_cv, RF_ind = get_clf_res_ML(devset, indset, method='RF')
    LinSVM_cv, LinSVM_ind = get_clf_res_ML(devset, indset, method='LinSVM')
    RBFSVM_cv, RBFSVM_ind = get_clf_res_ML(devset, indset, method='RBFSVM')
    MLP_cv, MLP_ind = get_clf_res_ML(devset, indset, method='MLP')
    
    fold_ind_perf = pd.concat([format_perf_mat(NB_ind, 'NB'), 
                               format_perf_mat(DT_ind, 'DT'), 
                               format_perf_mat(LR_ind, 'LR'),
                               format_perf_mat(RF_ind, 'RF'), 
                               format_perf_mat(LinSVM_ind, 'LinSVM'),
                               format_perf_mat(RBFSVM_ind, 'RBFSVM'),
                               format_perf_mat(MLP_ind, 'MLP')], axis=0)

    return fold_ind_perf

## Train and Evaluate Performance of Models

In [11]:
perf_raw_r6g_1 = get_all_clf_res(r6g_b1_raw, r6g_b2_raw, 'r6g_b1b2', 'raw')
perf_raw_r6g_2 = get_all_clf_res(r6g_b2_raw, r6g_b1_raw, 'r6g_b2b1', 'raw')

[50  0  1 99]
 
Training/ Validation / Test BACC : 0.998125 / 1.0 / 0.995  
Training/ Validation / Test Youden-index: 0.9962500000000001 / 1.0 / 0.99  
Training/ Validation / Test MCC: 0.99625 / 1.0 / 0.985184366143778  
Training/ Validation / Test AUC: 0.9987453125000001 / 1.0 / 1.0  
Training/ Validation / Test AUPR: 0.9993726620971954 / 1.0 / 1.0
[(0.668, 0.999, 0.006, 0.8004807692307693, 0.04570452650794567, 0.5025, 0.004999999999999893, 0.503502, 0.8339474903725441)]
[ 50   0   0 100]
 
Training/ Validation / Test BACC : 0.998125 / 1.0 / 1.0  
Training/ Validation / Test Youden-index: 0.9962500000000001 / 1.0 / 1.0  
Training/ Validation / Test MCC: 0.99625 / 1.0 / 1.0  
Training/ Validation / Test AUC: 0.99874375 / 1.0 / 1.0  
Training/ Validation / Test AUPR: 0.9993710995971953 / 1.0 / 1.0
[(0.668, 0.999, 0.006, 0.8004807692307693, 0.04570452650794567, 0.5025, 0.004999999999999893, 0.503502, 0.8339474903725441), (0.668, 0.999, 0.006, 0.8004807692307693, 0.04570452650794567, 0.50

[ 50   0   0 100]
 
Training/ Validation / Test BACC : 0.9975 / 1.0 / 1.0  
Training/ Validation / Test Youden-index: 0.9950000000000001 / 1.0 / 1.0  
Training/ Validation / Test MCC: 0.9943802591818176 / 1.0 / 1.0  
Training/ Validation / Test AUC: 0.99874375 / 1.0 / 1.0  
Training/ Validation / Test AUPR: 0.9993710995971953 / 1.0 / 1.0
[(0.668, 0.999, 0.006, 0.8004807692307693, 0.04570452650794567, 0.5025, 0.004999999999999893, 0.503502, 0.8339474903725441), (0.668, 0.999, 0.006, 0.8004807692307693, 0.04570452650794567, 0.5025, 0.004999999999999893, 0.5025029999999999, 0.8337243756675561), (0.6686666666666666, 0.999, 0.008, 0.8008016032064128, 0.05725025740766715, 0.5035, 0.006999999999999895, 0.503502, 0.8339474903725441), (0.668, 0.999, 0.006, 0.8004807692307693, 0.04570452650794567, 0.5025, 0.004999999999999893, 0.5025029999999999, 0.8337243756675561), (0.668, 0.999, 0.006, 0.8004807692307693, 0.04570452650794567, 0.5025, 0.004999999999999893, 0.5025029999999999, 0.833724375667556

/mnt/sypark0215/anaconda3/envs/sersnet/lib/python3.8/site-packages/sklearn/metrics/_classification.py:846: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)


[ 50   0   0 100]
 
Training/ Validation / Test BACC : 1.0 / 1.0 / 1.0  
Training/ Validation / Test Youden-index: 1.0 / 1.0 / 1.0  
Training/ Validation / Test MCC: 1.0 / 1.0 / 1.0  
Training/ Validation / Test AUC: 1.0 / 1.0 / 1.0  
Training/ Validation / Test AUPR: 1.0 / 1.0 / 1.0
[(0.6666666666666666, 1.0, 0.0, 0.8, 0.0, 0.5, 0.0, 0.5, 0.8333333333333333), (0.6666666666666666, 1.0, 0.0, 0.8, 0.0, 0.5, 0.0, 0.5, 0.8333333333333333)]


/mnt/sypark0215/anaconda3/envs/sersnet/lib/python3.8/site-packages/sklearn/metrics/_classification.py:846: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)


...

/mnt/sypark0215/anaconda3/envs/sersnet/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:582: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/mnt/sypark0215/anaconda3/envs/sersnet/lib/python3.8/site-packages/sklearn/metrics/_classification.py:846: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)


[49  1  1 99]
 
Training/ Validation / Test BACC : 0.99 / 0.97 / 0.985  
Training/ Validation / Test Youden-index: 0.98 / 0.94 / 0.97  
Training/ Validation / Test MCC: 0.9812334497691676 / 0.94 / 0.97  
Training/ Validation / Test AUC: 0.9975625000000001 / 0.9972 / 0.9995999999999999  
Training/ Validation / Test AUPR: 0.9989201737481201 / 0.9985551805545578 / 0.9998029508833237
[(0.666, 0.999, 0.0, 0.7995198079231692, -0.018263507434294623, 0.4995, -0.0010000000000000009, 0.47444400000000003, 0.7770305481487232), (0.6666666666666666, 1.0, 0.0, 0.8, 0.0, 0.5, 0.0, 0.485272, 0.7849425850628511), (0.6666666666666666, 1.0, 0.0, 0.8, 0.0, 0.5, 0.0, 0.488476, 0.7873788479261066), (0.6666666666666666, 1.0, 0.0, 0.8, 0.0, 0.5, 0.0, 0.4698189999999999, 0.7734120232100622), (0.666, 0.999, 0.0, 0.7995198079231692, -0.018263507434294623, 0.4995, -0.0010000000000000009, 0.47924199999999995, 0.7802396299844184), (0.666, 0.999, 0.0, 0.7995198079231692, -0.018263507434294623, 0.4995, -0.001000000000

/mnt/sypark0215/anaconda3/envs/sersnet/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:582: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/mnt/sypark0215/anaconda3/envs/sersnet/lib/python3.8/site-packages/sklearn/metrics/_classification.py:846: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)


[50  0  1 99]
 
Training/ Validation / Test BACC : 0.990625 / 0.99 / 0.995  
Training/ Validation / Test Youden-index: 0.98125 / 0.98 / 0.99  
Training/ Validation / Test MCC: 0.979429036553259 / 0.9850365626224087 / 0.985184366143778  
Training/ Validation / Test AUC: 0.9977437499999999 / 0.9974 / 1.0  
Training/ Validation / Test AUPR: 0.9989742052814821 / 0.9986221370581028 / 1.0
[(0.666, 0.999, 0.0, 0.7995198079231692, -0.018263507434294623, 0.4995, -0.0010000000000000009, 0.47444400000000003, 0.7770305481487232), (0.6666666666666666, 1.0, 0.0, 0.8, 0.0, 0.5, 0.0, 0.485272, 0.7849425850628511), (0.6666666666666666, 1.0, 0.0, 0.8, 0.0, 0.5, 0.0, 0.488476, 0.7873788479261066), (0.6666666666666666, 1.0, 0.0, 0.8, 0.0, 0.5, 0.0, 0.4698189999999999, 0.7734120232100622), (0.666, 0.999, 0.0, 0.7995198079231692, -0.018263507434294623, 0.4995, -0.0010000000000000009, 0.47924199999999995, 0.7802396299844184), (0.666, 0.999, 0.0, 0.7995198079231692, -0.018263507434294623, 0.4995, -0.001000000

/mnt/sypark0215/anaconda3/envs/sersnet/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:582: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/mnt/sypark0215/anaconda3/envs/sersnet/lib/python3.8/site-packages/sklearn/metrics/_classification.py:846: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)


In [12]:
perf_raw_r6g_1.groupby('Model').mean()

,Accuracy,Sensitivity,Specificity,F1,MCC,BACC,Yoden Index,AUROC,AUPR
Model,,,,,,,,,
DT,0.673667,0.7701,0.4808,0.745467,0.245111,0.62545,0.2509,0.625450,0.869878
LR,0.666667,1.0000,0.0000,0.800000,0.000000,0.50000,0.0000,0.182382,0.546086
LinSVM,0.666667,1.0000,0.0000,0.800000,0.000000,0.50000,0.0000,0.132995,0.520621
MLP,0.666667,1.0000,0.0000,0.800000,0.000000,0.50000,0.0000,0.389154,0.659813
NB,0.668133,0.9990,0.0064,0.800545,0.048014,0.50270,0.0054,0.503102,0.833858
RBFSVM,0.666667,1.0000,0.0000,0.800000,0.000000,0.50000,0.0000,0.305516,0.554548
RF,0.668333,0.9991,0.0068,0.800657,0.051911,0.50295,0.0059,0.800987,0.907787


In [13]:
perf_raw_r6g_2.groupby('Model').mean()

,Accuracy,Sensitivity,Specificity,F1,MCC,BACC,Yoden Index,AUROC,AUPR
Model,,,,,,,,,
DT,0.839933,0.9331,0.6536,0.888684,0.623054,0.79335,0.5867,0.793350,0.917498
LR,0.674933,0.9859,0.0530,0.801801,0.091485,0.51945,0.0389,0.553577,0.766058
LinSVM,0.665667,0.9981,0.0008,0.799215,-0.014745,0.49945,-0.0011,0.429970,0.791905
MLP,0.666467,0.9997,0.0000,0.799856,-0.005479,0.49985,-0.0003,0.482106,0.782606
NB,0.469400,0.7041,0.0000,0.637472,-0.349390,0.35205,-0.2959,0.349850,0.741924
RBFSVM,0.845600,0.8207,0.8954,0.876296,0.684646,0.85805,0.7161,0.905956,0.953487
RF,0.920000,0.9469,0.8662,0.941131,0.819346,0.90655,0.8131,0.959551,0.985511


In [14]:
pd.concat([perf_raw_r6g_1, perf_raw_r6g_2], axis=0).groupby('Model').mean()

,Accuracy,Sensitivity,Specificity,F1,MCC,BACC,Yoden Index,AUROC,AUPR
Model,,,,,,,,,
DT,0.756800,0.85160,0.5672,0.817076,0.434082,0.709400,0.41880,0.709400,0.893688
LR,0.670800,0.99295,0.0265,0.800901,0.045742,0.509725,0.01945,0.367980,0.656072
LinSVM,0.666167,0.99905,0.0004,0.799608,-0.007372,0.499725,-0.00055,0.281483,0.656263
MLP,0.666567,0.99985,0.0000,0.799928,-0.002740,0.499925,-0.00015,0.435630,0.721210
NB,0.568767,0.85155,0.0032,0.719009,-0.150688,0.427375,-0.14525,0.426476,0.787891
RBFSVM,0.756133,0.91035,0.4477,0.838148,0.342323,0.679025,0.35805,0.605736,0.754017
RF,0.794167,0.97300,0.4365,0.870894,0.435628,0.704750,0.40950,0.880269,0.946649


In [15]:
perf_psn_r6g_1 = get_all_clf_res(r6g_b1_psn, r6g_b2_psn, 'r6g_b1b2', 'psn')
perf_psn_r6g_2 = get_all_clf_res(r6g_b2_psn, r6g_b1_psn, 'r6g_b2b1', 'psn')

[ 50   0   0 100]
 
Training/ Validation / Test BACC : 0.99875 / 1.0 / 1.0  
Training/ Validation / Test Youden-index: 0.9975 / 1.0 / 1.0  
Training/ Validation / Test MCC: 0.9981255841064991 / 1.0 / 1.0  
Training/ Validation / Test AUC: 0.99875 / 1.0 / 1.0  
Training/ Validation / Test AUPR: 0.9993757802746567 / 1.0 / 1.0
[(0.6666666666666666, 1.0, 0.0, 0.8, 0.0, 0.5, 0.0, 0.5, 0.8333333333333333)]


/mnt/sypark0215/anaconda3/envs/sersnet/lib/python3.8/site-packages/sklearn/metrics/_classification.py:846: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)


[ 50   0   0 100]
 
Training/ Validation / Test BACC : 0.99875 / 1.0 / 1.0  
Training/ Validation / Test Youden-index: 0.9975 / 1.0 / 1.0  
Training/ Validation / Test MCC: 0.9981255841064991 / 1.0 / 1.0  
Training/ Validation / Test AUC: 0.99875 / 1.0 / 1.0  
Training/ Validation / Test AUPR: 0.9993757802746567 / 1.0 / 1.0
[(0.6666666666666666, 1.0, 0.0, 0.8, 0.0, 0.5, 0.0, 0.5, 0.8333333333333333), (0.6666666666666666, 1.0, 0.0, 0.8, 0.0, 0.5, 0.0, 0.5, 0.8333333333333333)]


/mnt/sypark0215/anaconda3/envs/sersnet/lib/python3.8/site-packages/sklearn/metrics/_classification.py:846: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)


[ 49   1   0 100]
 
Training/ Validation / Test BACC : 1.0 / 1.0 / 0.99  
Training/ Validation / Test Youden-index: 1.0 / 1.0 / 0.98  
Training/ Validation / Test MCC: 1.0 / 1.0 / 0.9850365626224087  
Training/ Validation / Test AUC: 1.0 / 1.0 / 0.99  
Training/ Validation / Test AUPR: 1.0 / 1.0 / 0.995049504950495
[(0.6666666666666666, 1.0, 0.0, 0.8, 0.0, 0.5, 0.0, 0.5, 0.8333333333333333), (0.6666666666666666, 1.0, 0.0, 0.8, 0.0, 0.5, 0.0, 0.5, 0.8333333333333333), (0.6666666666666666, 1.0, 0.0, 0.8, 0.0, 0.5, 0.0, 0.5, 0.8333333333333333)]


...

[ 48   2   0 100]
 
Training/ Validation / Test BACC : 0.9775 / 0.97 / 0.98  
Training/ Validation / Test Youden-index: 0.9550000000000001 / 0.94 / 0.96  
Training/ Validation / Test MCC: 0.9644266253272915 / 0.9553121789359229 / 0.9701425001453319  
Training/ Validation / Test AUC: 0.9970125 / 1.0 / 1.0  
Training/ Validation / Test AUPR: 0.9985950364661624 / 1.0 / 1.0
[(0.33266666666666667, 0.0, 0.998, 0.0, -0.036527014868589246, 0.499, -0.0020000000000000018, 0.014226000000000003, 0.4520848129134677), (0.33266666666666667, 0.0, 0.998, 0.0, -0.036527014868589246, 0.499, -0.0020000000000000018, 0.027054, 0.4554089792465297), (0.33266666666666667, 0.0, 0.998, 0.0, -0.036527014868589246, 0.499, -0.0020000000000000018, 0.01925, 0.45277203373745106), (0.33266666666666667, 0.0, 0.998, 0.0, -0.036527014868589246, 0.499, -0.0020000000000000018, 0.013566, 0.45203251820163454), (0.33266666666666667, 0.0, 0.998, 0.0, -0.036527014868589246, 0.499, -0.0020000000000000018, 0.010967999999999999, 0.

/mnt/sypark0215/anaconda3/envs/sersnet/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:582: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


[ 45   5   0 100]
 
Training/ Validation / Test BACC : 0.9450000000000001 / 0.95 / 0.95  
Training/ Validation / Test Youden-index: 0.8900000000000001 / 0.8999999999999999 / 0.8999999999999999  
Training/ Validation / Test MCC: 0.916262206989307 / 0.9258200997725515 / 0.9258200997725515  
Training/ Validation / Test AUC: 0.996196875 / 1.0 / 1.0  
Training/ Validation / Test AUPR: 0.9981229485664861 / 1.0 / 1.0
[(0.33266666666666667, 0.0, 0.998, 0.0, -0.036527014868589246, 0.499, -0.0020000000000000018, 0.014226000000000003, 0.4520848129134677), (0.33266666666666667, 0.0, 0.998, 0.0, -0.036527014868589246, 0.499, -0.0020000000000000018, 0.027054, 0.4554089792465297), (0.33266666666666667, 0.0, 0.998, 0.0, -0.036527014868589246, 0.499, -0.0020000000000000018, 0.01925, 0.45277203373745106), (0.33266666666666667, 0.0, 0.998, 0.0, -0.036527014868589246, 0.499, -0.0020000000000000018, 0.013566, 0.45203251820163454), (0.33266666666666667, 0.0, 0.998, 0.0, -0.036527014868589246, 0.499, -0.0020

/mnt/sypark0215/anaconda3/envs/sersnet/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:582: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


[ 49   1   0 100]
 
Training/ Validation / Test BACC : 0.985625 / 0.98 / 0.99  
Training/ Validation / Test Youden-index: 0.97125 / 0.96 / 0.98  
Training/ Validation / Test MCC: 0.9682665860655965 / 0.9553302944424514 / 0.9850365626224087  
Training/ Validation / Test AUC: 0.9971875 / 0.9874 / 1.0  
Training/ Validation / Test AUPR: 0.9983693882043256 / 0.9951205040160187 / 1.0
[(0.33266666666666667, 0.0, 0.998, 0.0, -0.036527014868589246, 0.499, -0.0020000000000000018, 0.014226000000000003, 0.4520848129134677), (0.33266666666666667, 0.0, 0.998, 0.0, -0.036527014868589246, 0.499, -0.0020000000000000018, 0.027054, 0.4554089792465297), (0.33266666666666667, 0.0, 0.998, 0.0, -0.036527014868589246, 0.499, -0.0020000000000000018, 0.01925, 0.45277203373745106), (0.33266666666666667, 0.0, 0.998, 0.0, -0.036527014868589246, 0.499, -0.0020000000000000018, 0.013566, 0.45203251820163454), (0.33266666666666667, 0.0, 0.998, 0.0, -0.036527014868589246, 0.499, -0.0020000000000000018, 0.0109679999999

In [16]:
perf_psn_r6g_1.groupby('Model').mean()

,Accuracy,Sensitivity,Specificity,F1,MCC,BACC,Yoden Index,AUROC,AUPR
Model,,,,,,,,,
DT,0.500133,0.2502,1.0000,0.333511,0.250150,0.62510,0.2502,0.625100,0.875033
LR,0.666667,1.0000,0.0000,0.800000,0.000000,0.50000,0.0000,0.236495,0.549180
LinSVM,0.666667,1.0000,0.0000,0.800000,0.000000,0.50000,0.0000,0.091170,0.466421
MLP,0.609533,0.7871,0.2544,0.716803,0.015612,0.52075,0.0415,0.531009,0.806225
NB,0.666667,1.0000,0.0000,0.800000,0.000000,0.50000,0.0000,0.500000,0.833333
RBFSVM,0.666667,1.0000,0.0000,0.800000,0.000000,0.50000,0.0000,0.549552,0.810176
RF,0.666667,1.0000,0.0000,0.800000,0.000000,0.50000,0.0000,0.763098,0.911377


In [17]:
perf_psn_r6g_2.groupby('Model').mean()

,Accuracy,Sensitivity,Specificity,F1,MCC,BACC,Yoden Index,AUROC,AUPR
Model,,,,,,,,,
DT,0.466867,0.3505,0.6996,0.321165,0.061421,0.52505,0.0501,0.525050,0.808208
LR,0.333333,0.0000,1.0000,0.000000,0.000000,0.50000,0.0000,0.206337,0.577830
LinSVM,0.333333,0.0000,1.0000,0.000000,0.000000,0.50000,0.0000,0.499800,0.716617
MLP,0.332667,0.0000,0.9980,0.000000,-0.036527,0.49900,-0.0020,0.015276,0.452467
NB,0.666667,1.0000,0.0000,0.800000,0.000000,0.50000,0.0000,0.500000,0.833333
RBFSVM,0.666667,1.0000,0.0000,0.800000,0.000000,0.50000,0.0000,0.045831,0.464178
RF,0.633733,0.8983,0.1046,0.719957,-0.004255,0.50145,0.0029,0.466312,0.661747


In [18]:
pd.concat([perf_psn_r6g_1, perf_psn_r6g_2], axis=0).groupby('Model').mean()

,Accuracy,Sensitivity,Specificity,F1,MCC,BACC,Yoden Index,AUROC,AUPR
Model,,,,,,,,,
DT,0.483500,0.30035,0.8498,0.327338,0.155785,0.575075,0.15015,0.575075,0.841621
LR,0.500000,0.50000,0.5000,0.400000,0.000000,0.500000,0.00000,0.221416,0.563505
LinSVM,0.500000,0.50000,0.5000,0.400000,0.000000,0.500000,0.00000,0.295485,0.591519
MLP,0.471100,0.39355,0.6262,0.358401,-0.010458,0.509875,0.01975,0.273142,0.629346
NB,0.666667,1.00000,0.0000,0.800000,0.000000,0.500000,0.00000,0.500000,0.833333
RBFSVM,0.666667,1.00000,0.0000,0.800000,0.000000,0.500000,0.00000,0.297691,0.637177
RF,0.650200,0.94915,0.0523,0.759979,-0.002128,0.500725,0.00145,0.614705,0.786562


In [19]:
perf_bn_r6g_1 = get_all_clf_res(r6g_b1_bn, r6g_b2_bn, 'r6g_b1b2', 'bn')
perf_bn_r6g_2 = get_all_clf_res(r6g_b2_bn, r6g_b1_bn, 'r6g_b2b1', 'bn')

[50  0 45 55]
 
Training/ Validation / Test BACC : 0.794375 / 0.7949999999999999 / 0.775  
Training/ Validation / Test Youden-index: 0.5887500000000001 / 0.5899999999999999 / 0.55  
Training/ Validation / Test MCC: 0.5677386073435892 / 0.5693644036781929 / 0.5380275868489703  
Training/ Validation / Test AUC: 0.8132312499999999 / 0.8213 / 0.79  
Training/ Validation / Test AUPR: 0.9368916598495538 / 0.9388624708624709 / 0.9299999999999999
[(0.798, 0.716, 0.962, 0.8253602305475505, 0.6393524134649383, 0.839, 0.6779999999999999, 0.862072, 0.947076406077017)]
[49  1 45 55]
 
Training/ Validation / Test BACC : 0.795625 / 0.77 / 0.765  
Training/ Validation / Test Youden-index: 0.59125 / 0.54 / 0.53  
Training/ Validation / Test MCC: 0.5697356343677193 / 0.5303300858899106 / 0.5165387788253609  
Training/ Validation / Test AUC: 0.8124562500000001 / 0.7858 / 0.7747999999999998  
Training/ Validation / Test AUPR: 0.9368511098685699 / 0.9264406779661016 / 0.9193647912885662
[(0.798, 0.716, 0.9

/mnt/sypark0215/anaconda3/envs/sersnet/lib/python3.8/site-packages/sklearn/metrics/_classification.py:846: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)


[48  2 43 57]
 
Training/ Validation / Test BACC : 0.798125 / 0.7849999999999999 / 0.7649999999999999  
Training/ Validation / Test Youden-index: 0.59625 / 0.5699999999999998 / 0.5299999999999998  
Training/ Validation / Test MCC: 0.5743479073455384 / 0.5535807194106618 / 0.5114629388973597  
Training/ Validation / Test AUC: 0.8129359375000001 / 0.8 / 0.7745999999999998  
Training/ Validation / Test AUPR: 0.9369869924464705 / 0.9333333333333333 / 0.9146175789571177
[(0.798, 0.716, 0.962, 0.8253602305475505, 0.6393524134649383, 0.839, 0.6779999999999999, 0.862072, 0.947076406077017), (0.6666666666666666, 1.0, 0.0, 0.8, 0.0, 0.5, 0.0, 0.5, 0.8333333333333333), (0.6666666666666666, 1.0, 0.0, 0.8, 0.0, 0.5, 0.0, 0.5, 0.8333333333333333)]


/mnt/sypark0215/anaconda3/envs/sersnet/lib/python3.8/site-packages/sklearn/metrics/_classification.py:846: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)


[50  0 46 54]
 
Training/ Validation / Test BACC : 0.7918750000000001 / 0.8 / 0.77  
Training/ Validation / Test Youden-index: 0.5837500000000002 / 0.6000000000000001 / 0.54  
Training/ Validation / Test MCC: 0.5637564499744441 / 0.5773502691896257 / 0.5303300858899106  
Training/ Validation / Test AUC: 0.8087046875000001 / 0.8 / 0.79  
Training/ Validation / Test AUPR: 0.9355855561575702 / 0.9333333333333333 / 0.9299999999999999
[(0.798, 0.716, 0.962, 0.8253602305475505, 0.6393524134649383, 0.839, 0.6779999999999999, 0.862072, 0.947076406077017), (0.6666666666666666, 1.0, 0.0, 0.8, 0.0, 0.5, 0.0, 0.5, 0.8333333333333333), (0.6666666666666666, 1.0, 0.0, 0.8, 0.0, 0.5, 0.0, 0.5, 0.8333333333333333), (0.8233333333333334, 0.757, 0.956, 0.8510399100618324, 0.6727259347714155, 0.8565, 0.7130000000000001, 0.8769520000000001, 0.9512833722354236)]
[50  0 38 62]
 
Training/ Validation / Test BACC : 0.79 / 0.7949999999999999 / 0.81  
Training/ Validation / Test Youden-index: 0.5800000000000001 /

...

[ 50   0   0 100]
 
Training/ Validation / Test BACC : 1.0 / 0.99 / 1.0  
Training/ Validation / Test Youden-index: 1.0 / 0.98 / 1.0  
Training/ Validation / Test MCC: 1.0 / 0.9850365626224087 / 1.0  
Training/ Validation / Test AUC: 1.0 / 1.0 / 1.0  
Training/ Validation / Test AUPR: 1.0 / 1.0 / 1.0
[(0.7106666666666667, 0.65, 0.832, 0.7497116493656286, 0.4545374023246612, 0.741, 0.482, 0.78945, 0.9079136930668222), (0.74, 0.667, 0.886, 0.7737819025522041, 0.521686970137444, 0.7765, 0.5529999999999999, 0.842846, 0.9284594156787811), (0.714, 0.614, 0.914, 0.7410983705491853, 0.501674992220331, 0.764, 0.528, 0.772376, 0.902444466620065), (0.716, 0.648, 0.852, 0.7526132404181185, 0.47173338117241753, 0.75, 0.5, 0.783291, 0.9056280203697382), (0.7133333333333334, 0.625, 0.89, 0.7440476190476191, 0.48767539978182955, 0.7575000000000001, 0.5150000000000001, 0.793284, 0.9090813720848068), (0.6986666666666667, 0.642, 0.812, 0.7396313364055299, 0.428109897634402, 0.7270000000000001, 0.45400000

[(0.866, 0.838, 0.922, 0.8929142248268513, 0.7270340890071023, 0.88, 0.76, 0.9542660000000001, 0.9783492643564474), (0.858, 0.828, 0.918, 0.8860353130016051, 0.7123595678788633, 0.873, 0.746, 0.9488380000000001, 0.9756464680874921), (0.8446666666666667, 0.822, 0.89, 0.8758657432072456, 0.6811161465434958, 0.856, 0.712, 0.9361090000000001, 0.9697952756939769), (0.8733333333333333, 0.845, 0.93, 0.898936170212766, 0.7419070941591247, 0.8875, 0.7749999999999999, 0.959932, 0.9809560174950961)]
[ 50   0   0 100]
 
Training/ Validation / Test BACC : 1.0 / 1.0 / 1.0  
Training/ Validation / Test Youden-index: 1.0 / 1.0 / 1.0  
Training/ Validation / Test MCC: 1.0 / 1.0 / 1.0  
Training/ Validation / Test AUC: 1.0 / 1.0 / 1.0  
Training/ Validation / Test AUPR: 1.0 / 1.0 / 1.0
[(0.866, 0.838, 0.922, 0.8929142248268513, 0.7270340890071023, 0.88, 0.76, 0.9542660000000001, 0.9783492643564474), (0.858, 0.828, 0.918, 0.8860353130016051, 0.7123595678788633, 0.873, 0.746, 0.9488380000000001, 0.9756464

[(0.8533333333333334, 0.802, 0.956, 0.8793859649122807, 0.7181534546620522, 0.879, 0.758, 0.9683539999999999, 0.9839042987901553)]
[ 50   0   0 100]
 
Training/ Validation / Test BACC : 0.975625 / 0.96 / 1.0  
Training/ Validation / Test Youden-index: 0.9512499999999999 / 0.9199999999999999 / 1.0  
Training/ Validation / Test MCC: 0.9626342856215807 / 0.9247282016455265 / 1.0  
Training/ Validation / Test AUC: 0.9796499999999999 / 0.9613999999999999 / 1.0  
Training/ Validation / Test AUPR: 0.9797177309939776 / 0.9576890468870587 / 1.0
[(0.8533333333333334, 0.802, 0.956, 0.8793859649122807, 0.7181534546620522, 0.879, 0.758, 0.9683539999999999, 0.9839042987901553), (0.8526666666666667, 0.802, 0.954, 0.8789041095890411, 0.716354406954851, 0.878, 0.756, 0.9684119999999999, 0.9838872011752029)]
[ 47   3   0 100]
 
Training/ Validation / Test BACC : 0.9750000000000001 / 0.99 / 0.97  
Training/ Validation / Test Youden-index: 0.9500000000000002 / 0.98 / 0.94  
Training/ Validation / Test MCC

[47  3  2 98]
 
Training/ Validation / Test BACC : 0.97875 / 0.97 / 0.96  
Training/ Validation / Test Youden-index: 0.9575 / 0.94 / 0.9199999999999999  
Training/ Validation / Test MCC: 0.968285339860108 / 0.9553121789359229 / 0.9247282016455265  
Training/ Validation / Test AUC: 0.98076875 / 0.9842 / 0.9705999999999999  
Training/ Validation / Test AUPR: 0.9800261630256939 / 0.9882902333969106 / 0.9752537216218611
[(0.8533333333333334, 0.802, 0.956, 0.8793859649122807, 0.7181534546620522, 0.879, 0.758, 0.9683539999999999, 0.9839042987901553), (0.8526666666666667, 0.802, 0.954, 0.8789041095890411, 0.716354406954851, 0.878, 0.756, 0.9684119999999999, 0.9838872011752029), (0.8546666666666667, 0.804, 0.956, 0.8806133625410734, 0.7202422975445439, 0.88, 0.76, 0.968741, 0.984080915169449), (0.8646666666666667, 0.821, 0.952, 0.8899728997289973, 0.734709222869787, 0.8865, 0.7729999999999999, 0.9715999999999999, 0.9854854769843157), (0.858, 0.811, 0.952, 0.8839237057220708, 0.7240281966186792

[ 50   0   0 100]
 
Training/ Validation / Test BACC : 1.0 / 0.99 / 1.0  
Training/ Validation / Test Youden-index: 1.0 / 0.98 / 1.0  
Training/ Validation / Test MCC: 1.0 / 0.9850365626224087 / 1.0  
Training/ Validation / Test AUC: 1.0 / 1.0 / 1.0  
Training/ Validation / Test AUPR: 1.0 / 1.0 / 1.0
[(0.9646666666666667, 0.971, 0.952, 0.9734335839598998, 0.920724021094837, 0.9615, 0.923, 0.9938579999999999, 0.9967772754416657), (0.9646666666666667, 0.969, 0.956, 0.9733802109492717, 0.9209393214709632, 0.9624999999999999, 0.9249999999999998, 0.993384, 0.9965703587249302), (0.966, 0.971, 0.956, 0.9744104365278474, 0.9238173883460375, 0.9635, 0.927, 0.993596, 0.9966577615717083), (0.954, 0.955, 0.952, 0.965133906013138, 0.8980118791576709, 0.9535, 0.907, 0.992022, 0.9958838888510788), (0.9626666666666667, 0.967, 0.954, 0.9718592964824121, 0.9165200012307269, 0.9604999999999999, 0.9209999999999998, 0.993236, 0.9964702495916001), (0.9533333333333334, 0.959, 0.942, 0.9647887323943662, 0.895

In [20]:
perf_bn_r6g_1.groupby('Model').mean()

,Accuracy,Sensitivity,Specificity,F1,MCC,BACC,Yoden Index,AUROC,AUPR
Model,,,,,,,,,
DT,0.883133,0.9356,0.7782,0.914404,0.734509,0.85690,0.7138,0.856900,0.936908
LR,0.898533,1.0000,0.6956,0.929365,0.776942,0.84780,0.6956,0.980311,0.984954
LinSVM,0.902600,0.9941,0.7196,0.932866,0.782992,0.85685,0.7137,0.973383,0.983019
MLP,0.959067,0.9999,0.8774,0.970231,0.909092,0.93865,0.8773,0.983451,0.990022
NB,0.775200,0.8291,0.6674,0.834707,0.468609,0.74825,0.4965,0.763668,0.915736
RBFSVM,0.953400,0.9716,0.9170,0.965171,0.895477,0.94430,0.8886,0.981645,0.989332
RF,0.720800,0.5974,0.9676,0.731993,0.555867,0.78250,0.5650,0.976084,0.985625


In [21]:
perf_bn_r6g_2.groupby('Model').mean()

,Accuracy,Sensitivity,Specificity,F1,MCC,BACC,Yoden Index,AUROC,AUPR
Model,,,,,,,,,
DT,0.736667,0.6808,0.8484,0.772432,0.506539,0.76460,0.5292,0.764600,0.897582
LR,0.872600,0.8341,0.9496,0.897220,0.746867,0.89185,0.7837,0.964443,0.983033
LinSVM,0.862667,0.8355,0.9170,0.890251,0.719876,0.87625,0.7525,0.953023,0.977679
MLP,0.959400,0.9655,0.9472,0.969415,0.909196,0.95635,0.9127,0.992517,0.996097
NB,0.808667,0.7778,0.8704,0.844219,0.615993,0.82410,0.6482,0.832965,0.925393
RBFSVM,0.857733,0.8099,0.9534,0.883567,0.724195,0.88165,0.7633,0.969295,0.984264
RF,0.720467,0.6462,0.8690,0.754845,0.486809,0.75760,0.5152,0.802854,0.913054


In [22]:
pd.concat([perf_bn_r6g_1, perf_bn_r6g_2], axis=0).groupby('Model').mean()

,Accuracy,Sensitivity,Specificity,F1,MCC,BACC,Yoden Index,AUROC,AUPR
Model,,,,,,,,,
DT,0.809900,0.80820,0.8133,0.843418,0.620524,0.810750,0.62150,0.810750,0.917245
LR,0.885567,0.91705,0.8226,0.913292,0.761905,0.869825,0.73965,0.972377,0.983993
LinSVM,0.882633,0.91480,0.8183,0.911559,0.751434,0.866550,0.73310,0.963203,0.980349
MLP,0.959233,0.98270,0.9123,0.969823,0.909144,0.947500,0.89500,0.987984,0.993059
NB,0.791933,0.80345,0.7689,0.839463,0.542301,0.786175,0.57235,0.798317,0.920564
RBFSVM,0.905567,0.89075,0.9352,0.924369,0.809836,0.912975,0.82595,0.975470,0.986798
RF,0.720633,0.62180,0.9183,0.743419,0.521338,0.770050,0.54010,0.889469,0.949340


In [23]:
perf_bc_r6g_1 = get_all_clf_res(r6g_b1_bl, r6g_b2_bl, 'r6g_b1b2', 'bc')
perf_bc_r6g_2 = get_all_clf_res(r6g_b2_bl, r6g_b1_bl, 'r6g_b2b1', 'bc')

[ 49   1   0 100]
 
Training/ Validation / Test BACC : 0.9962500000000001 / 1.0 / 0.99  
Training/ Validation / Test Youden-index: 0.9925000000000002 / 1.0 / 0.98  
Training/ Validation / Test MCC: 0.994380224002831 / 1.0 / 0.9850365626224087  
Training/ Validation / Test AUC: 0.9962500000000001 / 1.0 / 0.99  
Training/ Validation / Test AUPR: 0.99813200498132 / 1.0 / 0.995049504950495
[(0.6666666666666666, 1.0, 0.0, 0.8, 0.0, 0.5, 0.0, 0.5, 0.8333333333333333)]


/mnt/sypark0215/anaconda3/envs/sersnet/lib/python3.8/site-packages/sklearn/metrics/_classification.py:846: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)


[ 50   0   0 100]
 
Training/ Validation / Test BACC : 0.99875 / 0.97 / 1.0  
Training/ Validation / Test Youden-index: 0.9975 / 0.94 / 1.0  
Training/ Validation / Test MCC: 0.9981255841064991 / 0.9553121789359229 / 1.0  
Training/ Validation / Test AUC: 0.99875 / 0.97 / 1.0  
Training/ Validation / Test AUPR: 0.9993757802746567 / 0.9854368932038835 / 1.0
[(0.6666666666666666, 1.0, 0.0, 0.8, 0.0, 0.5, 0.0, 0.5, 0.8333333333333333), (0.6666666666666666, 1.0, 0.0, 0.8, 0.0, 0.5, 0.0, 0.5, 0.8333333333333333)]


/mnt/sypark0215/anaconda3/envs/sersnet/lib/python3.8/site-packages/sklearn/metrics/_classification.py:846: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)


[ 50   0   0 100]
 
Training/ Validation / Test BACC : 0.995 / 1.0 / 1.0  
Training/ Validation / Test Youden-index: 0.99 / 1.0 / 1.0  
Training/ Validation / Test MCC: 0.9925092578236596 / 1.0 / 1.0  
Training/ Validation / Test AUC: 0.995 / 1.0 / 1.0  
Training/ Validation / Test AUPR: 0.9975124378109452 / 1.0 / 1.0
[(0.6666666666666666, 1.0, 0.0, 0.8, 0.0, 0.5, 0.0, 0.5, 0.8333333333333333), (0.6666666666666666, 1.0, 0.0, 0.8, 0.0, 0.5, 0.0, 0.5, 0.8333333333333333), (0.6666666666666666, 1.0, 0.0, 0.8, 0.0, 0.5, 0.0, 0.5, 0.8333333333333333)]


...

/mnt/sypark0215/anaconda3/envs/sersnet/lib/python3.8/site-packages/sklearn/metrics/_classification.py:846: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/mnt/sypark0215/anaconda3/envs/sersnet/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:582: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/mnt/sypark0215/anaconda3/envs/sersnet/lib/python3.8/site-packages/sklearn/metrics/_classification.py:846: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)


[ 50   0   0 100]
 
Training/ Validation / Test BACC : 1.0 / 0.995 / 1.0  
Training/ Validation / Test Youden-index: 1.0 / 0.99 / 1.0  
Training/ Validation / Test MCC: 1.0 / 0.985184366143778 / 1.0  
Training/ Validation / Test AUC: 1.0 / 1.0 / 1.0  
Training/ Validation / Test AUPR: 1.0 / 1.0000000000000002 / 1.0
[(0.6666666666666666, 1.0, 0.0, 0.8, 0.0, 0.5, 0.0, 0.26530600000000004, 0.6829893130437868), (0.6666666666666666, 1.0, 0.0, 0.8, 0.0, 0.5, 0.0, 0.314116, 0.7178670324205267), (0.6666666666666666, 1.0, 0.0, 0.8, 0.0, 0.5, 0.0, 0.27281299999999997, 0.6760668502950545), (0.6666666666666666, 1.0, 0.0, 0.8, 0.0, 0.5, 0.0, 0.284767, 0.6971484458530708), (0.6666666666666666, 1.0, 0.0, 0.8, 0.0, 0.5, 0.0, 0.30676299999999995, 0.7126996100890111), (0.6666666666666666, 1.0, 0.0, 0.8, 0.0, 0.5, 0.0, 0.27230000000000004, 0.6881156206953214), (0.6666666666666666, 1.0, 0.0, 0.8, 0.0, 0.5, 0.0, 0.35559999999999997, 0.7457992575982066), (0.6666666666666666, 1.0, 0.0, 0.8, 0.0, 0.5, 0.0, 0.

/mnt/sypark0215/anaconda3/envs/sersnet/lib/python3.8/site-packages/sklearn/metrics/_classification.py:846: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)


[ 49   1   0 100]
 
Training/ Validation / Test BACC : 1.0 / 0.985 / 0.99  
Training/ Validation / Test Youden-index: 1.0 / 0.97 / 0.98  
Training/ Validation / Test MCC: 1.0 / 0.97 / 0.9850365626224087  
Training/ Validation / Test AUC: 1.0 / 0.9938 / 1.0  
Training/ Validation / Test AUPR: 1.0 / 0.9976244862008221 / 1.0
[(0.6666666666666666, 1.0, 0.0, 0.8, 0.0, 0.5, 0.0, 0.26530600000000004, 0.6829893130437868), (0.6666666666666666, 1.0, 0.0, 0.8, 0.0, 0.5, 0.0, 0.314116, 0.7178670324205267), (0.6666666666666666, 1.0, 0.0, 0.8, 0.0, 0.5, 0.0, 0.27281299999999997, 0.6760668502950545), (0.6666666666666666, 1.0, 0.0, 0.8, 0.0, 0.5, 0.0, 0.284767, 0.6971484458530708), (0.6666666666666666, 1.0, 0.0, 0.8, 0.0, 0.5, 0.0, 0.30676299999999995, 0.7126996100890111), (0.6666666666666666, 1.0, 0.0, 0.8, 0.0, 0.5, 0.0, 0.27230000000000004, 0.6881156206953214), (0.6666666666666666, 1.0, 0.0, 0.8, 0.0, 0.5, 0.0, 0.35559999999999997, 0.7457992575982066), (0.6666666666666666, 1.0, 0.0, 0.8, 0.0, 0.5, 

/mnt/sypark0215/anaconda3/envs/sersnet/lib/python3.8/site-packages/sklearn/metrics/_classification.py:846: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)


In [24]:
perf_bc_r6g_1.groupby('Model').mean()

,Accuracy,Sensitivity,Specificity,F1,MCC,BACC,Yoden Index,AUROC,AUPR
Model,,,,,,,,,
DT,0.623067,0.9300,0.0092,0.766856,-0.130955,0.46960,-0.0608,0.469600,0.814545
LR,0.666667,1.0000,0.0000,0.800000,0.000000,0.50000,0.0000,0.330600,0.558535
LinSVM,0.666667,1.0000,0.0000,0.800000,0.000000,0.50000,0.0000,0.209718,0.511929
MLP,0.666667,1.0000,0.0000,0.800000,0.000000,0.50000,0.0000,0.139325,0.479592
NB,0.666667,1.0000,0.0000,0.800000,0.000000,0.50000,0.0000,0.500000,0.833333
RBFSVM,0.666667,1.0000,0.0000,0.800000,0.000000,0.50000,0.0000,0.458351,0.654970
RF,0.667000,0.9983,0.0044,0.799883,0.038974,0.50135,0.0027,0.846631,0.930729


In [25]:
perf_bc_r6g_2.groupby('Model').mean()

,Accuracy,Sensitivity,Specificity,F1,MCC,BACC,Yoden Index,AUROC,AUPR
Model,,,,,,,,,
DT,0.773333,0.9693,0.3814,0.857818,0.363180,0.67535,0.3507,0.675350,0.886975
LR,0.665800,0.9980,0.0014,0.799263,-0.007751,0.49970,-0.0006,0.598330,0.791833
LinSVM,0.666267,0.9994,0.0000,0.799712,-0.009889,0.49970,-0.0006,0.460925,0.810760
MLP,0.666667,1.0000,0.0000,0.800000,0.000000,0.50000,0.0000,0.311347,0.707690
NB,0.666667,1.0000,0.0000,0.800000,0.000000,0.50000,0.0000,0.500000,0.833333
RBFSVM,0.665667,0.9954,0.0062,0.798782,0.001463,0.50080,0.0016,0.920991,0.965062
RF,0.734867,0.9972,0.2102,0.835134,0.343847,0.60370,0.2074,0.987854,0.994876


In [26]:
pd.concat([perf_bc_r6g_1, perf_bc_r6g_2], axis=0).groupby('Model').mean()

,Accuracy,Sensitivity,Specificity,F1,MCC,BACC,Yoden Index,AUROC,AUPR
Model,,,,,,,,,
DT,0.698200,0.94965,0.1953,0.812337,0.116113,0.572475,0.14495,0.572475,0.850760
LR,0.666233,0.99900,0.0007,0.799632,-0.003876,0.499850,-0.00030,0.464465,0.675184
LinSVM,0.666467,0.99970,0.0000,0.799856,-0.004945,0.499850,-0.00030,0.335321,0.661345
MLP,0.666667,1.00000,0.0000,0.800000,0.000000,0.500000,0.00000,0.225336,0.593641
NB,0.666667,1.00000,0.0000,0.800000,0.000000,0.500000,0.00000,0.500000,0.833333
RBFSVM,0.666167,0.99770,0.0031,0.799391,0.000731,0.500400,0.00080,0.689671,0.810016
RF,0.700933,0.99775,0.1073,0.817509,0.191410,0.552525,0.10505,0.917243,0.962803


In [27]:
perf_bc_psn_r6g_1 = get_all_clf_res(r6g_b1_bl_psn, r6g_b2_bl_psn, 'r6g_b1b2', 'bc_psn')
perf_bc_psn_r6g_2 = get_all_clf_res(r6g_b2_bl_psn, r6g_b1_bl_psn, 'r6g_b2b1', 'bc_psn')

[ 50   0   0 100]
 
Training/ Validation / Test BACC : 0.99875 / 1.0 / 1.0  
Training/ Validation / Test Youden-index: 0.9975 / 1.0 / 1.0  
Training/ Validation / Test MCC: 0.9981255841064991 / 1.0 / 1.0  
Training/ Validation / Test AUC: 0.99875 / 1.0 / 1.0  
Training/ Validation / Test AUPR: 0.9993757802746567 / 1.0 / 1.0
[(0.6666666666666666, 1.0, 0.0, 0.8, 0.0, 0.5, 0.0, 0.5, 0.8333333333333333)]


/mnt/sypark0215/anaconda3/envs/sersnet/lib/python3.8/site-packages/sklearn/metrics/_classification.py:846: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)


[ 50   0   0 100]
 
Training/ Validation / Test BACC : 0.99875 / 1.0 / 1.0  
Training/ Validation / Test Youden-index: 0.9975 / 1.0 / 1.0  
Training/ Validation / Test MCC: 0.9981255841064991 / 1.0 / 1.0  
Training/ Validation / Test AUC: 0.99875 / 1.0 / 1.0  
Training/ Validation / Test AUPR: 0.9993757802746567 / 1.0 / 1.0
[(0.6666666666666666, 1.0, 0.0, 0.8, 0.0, 0.5, 0.0, 0.5, 0.8333333333333333), (0.6666666666666666, 1.0, 0.0, 0.8, 0.0, 0.5, 0.0, 0.5, 0.8333333333333333)]


/mnt/sypark0215/anaconda3/envs/sersnet/lib/python3.8/site-packages/sklearn/metrics/_classification.py:846: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)


[ 50   0   0 100]
 
Training/ Validation / Test BACC : 0.99875 / 1.0 / 1.0  
Training/ Validation / Test Youden-index: 0.9975 / 1.0 / 1.0  
Training/ Validation / Test MCC: 0.9981255841064991 / 1.0 / 1.0  
Training/ Validation / Test AUC: 0.99875 / 1.0 / 1.0  
Training/ Validation / Test AUPR: 0.9993757802746567 / 1.0 / 1.0
[(0.6666666666666666, 1.0, 0.0, 0.8, 0.0, 0.5, 0.0, 0.5, 0.8333333333333333), (0.6666666666666666, 1.0, 0.0, 0.8, 0.0, 0.5, 0.0, 0.5, 0.8333333333333333), (0.6666666666666666, 1.0, 0.0, 0.8, 0.0, 0.5, 0.0, 0.5, 0.8333333333333333)]


...

/mnt/sypark0215/anaconda3/envs/sersnet/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:582: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


[ 50   0   0 100]
 
Training/ Validation / Test BACC : 0.999375 / 1.0 / 1.0  
Training/ Validation / Test Youden-index: 0.99875 / 1.0 / 1.0  
Training/ Validation / Test MCC: 0.9981279234752164 / 1.0 / 1.0  
Training/ Validation / Test AUC: 0.9999875 / 1.0 / 1.0  
Training/ Validation / Test AUPR: 0.9999937772222325 / 1.0 / 1.0
[(0.15666666666666668, 0.235, 0.0, 0.27089337175792505, -0.7213932098830056, 0.1175, -0.765, 0.049682, 0.4563930938985183), (0.188, 0.282, 0.0, 0.3164983164983165, -0.677553897404087, 0.141, -0.718, 0.112556, 0.4777365199494762), (0.172, 0.258, 0.0, 0.2935153583617747, -0.6996041096864979, 0.129, -0.742, 0.05075400000000001, 0.45661070171834095), (0.166, 0.249, 0.0, 0.2847341337907376, -0.7080502196063823, 0.1245, -0.751, 0.051418000000000005, 0.45685780966028516), (0.16933333333333334, 0.254, 0.0, 0.2896237172177879, -0.7033455482280396, 0.127, -0.746, 0.030587999999999997, 0.4533027631966556), (0.17, 0.255, 0.0, 0.2905982905982906, -0.7024083452264136, 0.1275,

/mnt/sypark0215/anaconda3/envs/sersnet/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:582: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


[ 49   1   0 100]
 
Training/ Validation / Test BACC : 0.999375 / 1.0 / 0.99  
Training/ Validation / Test Youden-index: 0.99875 / 1.0 / 0.98  
Training/ Validation / Test MCC: 0.9981279234752164 / 1.0 / 0.9850365626224087  
Training/ Validation / Test AUC: 0.999996875 / 1.0 / 1.0  
Training/ Validation / Test AUPR: 0.9999984384753433 / 1.0 / 1.0
[(0.15666666666666668, 0.235, 0.0, 0.27089337175792505, -0.7213932098830056, 0.1175, -0.765, 0.049682, 0.4563930938985183), (0.188, 0.282, 0.0, 0.3164983164983165, -0.677553897404087, 0.141, -0.718, 0.112556, 0.4777365199494762), (0.172, 0.258, 0.0, 0.2935153583617747, -0.6996041096864979, 0.129, -0.742, 0.05075400000000001, 0.45661070171834095), (0.166, 0.249, 0.0, 0.2847341337907376, -0.7080502196063823, 0.1245, -0.751, 0.051418000000000005, 0.45685780966028516), (0.16933333333333334, 0.254, 0.0, 0.2896237172177879, -0.7033455482280396, 0.127, -0.746, 0.030587999999999997, 0.4533027631966556), (0.17, 0.255, 0.0, 0.2905982905982906, -0.702408

/mnt/sypark0215/anaconda3/envs/sersnet/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:582: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


In [28]:
perf_bc_psn_r6g_1.groupby('Model').mean()

,Accuracy,Sensitivity,Specificity,F1,MCC,BACC,Yoden Index,AUROC,AUPR
Model,,,,,,,,,
DT,0.341867,0.5079,0.0098,0.507129,-0.483040,0.25885,-0.4823,0.258850,0.671168
LR,0.334067,0.5011,0.0000,0.500825,-0.499175,0.25055,-0.4989,0.233428,0.518156
LinSVM,0.346000,0.5190,0.0000,0.514044,-0.485845,0.25950,-0.4810,0.118994,0.474696
MLP,0.666667,1.0000,0.0000,0.800000,0.000000,0.50000,0.0000,0.056291,0.460170
NB,0.666667,1.0000,0.0000,0.800000,0.000000,0.50000,0.0000,0.500000,0.833333
RBFSVM,0.666667,1.0000,0.0000,0.800000,0.000000,0.50000,0.0000,0.077554,0.462840
RF,0.340800,0.5096,0.0032,0.507557,-0.489197,0.25640,-0.4872,0.251012,0.607685


In [29]:
perf_bc_psn_r6g_2.groupby('Model').mean()

,Accuracy,Sensitivity,Specificity,F1,MCC,BACC,Yoden Index,AUROC,AUPR
Model,,,,,,,,,
DT,0.574533,0.7616,0.2004,0.670357,-0.015544,0.48100,-0.0380,0.481000,0.787872
LR,0.449333,0.1743,0.9994,0.289817,0.242924,0.58685,0.1737,0.600108,0.797579
LinSVM,0.669733,0.7607,0.4878,0.735498,0.294894,0.62425,0.2485,0.718524,0.841546
MLP,0.186800,0.2802,0.0000,0.313214,-0.680694,0.14010,-0.7198,0.074966,0.467436
NB,0.034533,0.0242,0.0552,0.032282,-0.922345,0.03970,-0.9206,0.028818,0.366303
RBFSVM,0.333333,0.0000,1.0000,0.000000,0.000000,0.50000,0.0000,0.291524,0.658374
RF,0.483600,0.2254,1.0000,0.361041,0.293566,0.61270,0.2254,0.496025,0.771696


In [30]:
pd.concat([perf_bc_psn_r6g_1, perf_bc_psn_r6g_2], axis=0).groupby('Model').mean()

,Accuracy,Sensitivity,Specificity,F1,MCC,BACC,Yoden Index,AUROC,AUPR
Model,,,,,,,,,
DT,0.458200,0.63475,0.1051,0.588743,-0.249292,0.369925,-0.26015,0.369925,0.729520
LR,0.391700,0.33770,0.4997,0.395321,-0.128126,0.418700,-0.16260,0.416768,0.657867
LinSVM,0.507867,0.63985,0.2439,0.624771,-0.095475,0.441875,-0.11625,0.418759,0.658121
MLP,0.426733,0.64010,0.0000,0.556607,-0.340347,0.320050,-0.35990,0.065628,0.463803
NB,0.350600,0.51210,0.0276,0.416141,-0.461173,0.269850,-0.46030,0.264409,0.599818
RBFSVM,0.500000,0.50000,0.5000,0.400000,0.000000,0.500000,0.00000,0.184539,0.560607
RF,0.412200,0.36750,0.5016,0.434299,-0.097816,0.434550,-0.13090,0.373519,0.689690


In [31]:
perf_bc_bn_r6g_1 = get_all_clf_res(r6g_b1_bl_bn, r6g_b2_bl_bn, 'r6g_b1b2', 'bc_bn')
perf_bc_bn_r6g_2 = get_all_clf_res(r6g_b2_bl_bn, r6g_b1_bl_bn, 'r6g_b2b1', 'bc_bn')

[50  0 10 90]
 
Training/ Validation / Test BACC : 0.933125 / 0.87 / 0.95  
Training/ Validation / Test Youden-index: 0.86625 / 0.74 / 0.8999999999999999  
Training/ Validation / Test MCC: 0.8353351483845166 / 0.7053058877882966 / 0.8660254037844386  
Training/ Validation / Test AUC: 0.9632296875 / 0.9044 / 0.9628  
Training/ Validation / Test AUPR: 0.9849527416287086 / 0.9571236846584404 / 0.9871674409434729
[(0.7073333333333334, 0.604, 0.914, 0.7334547662416515, 0.49304675632692974, 0.759, 0.518, 0.800802, 0.9095461058077088)]
[49  1 11 89]
 
Training/ Validation / Test BACC : 0.928125 / 0.94 / 0.935  
Training/ Validation / Test Youden-index: 0.85625 / 0.8799999999999999 / 0.8700000000000001  
Training/ Validation / Test MCC: 0.8225298799189772 / 0.8492214834522198 / 0.8371578903249574  
Training/ Validation / Test AUC: 0.9571625 / 0.9588999999999999 / 0.9635999999999999  
Training/ Validation / Test AUPR: 0.9824785313896738 / 0.9838898727985447 / 0.9853477049864028
[(0.707333333333

[47  3 16 84]
 
Training/ Validation / Test BACC : 0.925 / 0.935 / 0.8899999999999999  
Training/ Validation / Test Youden-index: 0.8500000000000001 / 0.8700000000000001 / 0.7799999999999998  
Training/ Validation / Test MCC: 0.8152047480596408 / 0.8309489698388166 / 0.7449887315796287  
Training/ Validation / Test AUC: 0.9595624999999999 / 0.97 / 0.9135  
Training/ Validation / Test AUPR: 0.9832183718665141 / 0.99 / 0.963713981108145
[(0.7073333333333334, 0.604, 0.914, 0.7334547662416515, 0.49304675632692974, 0.759, 0.518, 0.800802, 0.9095461058077088), (0.7053333333333334, 0.601, 0.914, 0.7311435523114356, 0.49046996232573625, 0.7575000000000001, 0.5150000000000001, 0.7984739999999999, 0.9086810169921213), (0.7113333333333334, 0.615, 0.904, 0.7396271797955503, 0.4926436245403035, 0.7595000000000001, 0.5190000000000001, 0.799022, 0.9077083771892817), (0.7133333333333334, 0.621, 0.898, 0.742822966507177, 0.49198578530274484, 0.7595000000000001, 0.5190000000000001, 0.80011, 0.9075376310

[47  3  1 99]
 
Training/ Validation / Test BACC : 1.0 / 0.985 / 0.965  
Training/ Validation / Test Youden-index: 1.0 / 0.97 / 0.9299999999999999  
Training/ Validation / Test MCC: 1.0 / 0.97 / 0.9398255470157904  
Training/ Validation / Test AUC: 1.0 / 0.985 / 0.9649999999999999  
Training/ Validation / Test AUPR: 1.0 / 0.9933333333333333 / 0.9836274509803922
[(0.27266666666666667, 0.331, 0.156, 0.37763833428408444, -0.4836649076663372, 0.2435, -0.513, 0.24350000000000002, 0.6082875166002657), (0.6433333333333333, 0.962, 0.006, 0.7824318828792192, -0.09251569297275711, 0.484, -0.03200000000000003, 0.484, 0.8233445282156728), (0.48, 0.718, 0.004, 0.648014440433213, -0.3345059449476755, 0.361, -0.278, 0.361, 0.7482302631578946), (0.63, 0.94, 0.01, 0.7720739219712526, -0.11576369428032277, 0.475, -0.050000000000000044, 0.475, 0.8175261324041811), (0.608, 0.911, 0.002, 0.7560165975103734, -0.17269248691086997, 0.4565, -0.08699999999999997, 0.4565, 0.8082163120567375), (0.68, 0.884, 0.272

[ 50   0   0 100]
 
Training/ Validation / Test BACC : 1.0 / 1.0 / 1.0  
Training/ Validation / Test Youden-index: 1.0 / 1.0 / 1.0  
Training/ Validation / Test MCC: 1.0 / 1.0 / 1.0  
Training/ Validation / Test AUC: 1.0 / 1.0 / 1.0  
Training/ Validation / Test AUPR: 1.0 / 1.0 / 1.0
[(0.6246666666666667, 0.526, 0.822, 0.6513931888544892, 0.33354547472912566, 0.6739999999999999, 0.34799999999999986, 0.6283500000000001, 0.841670388003335), (0.6466666666666666, 0.515, 0.91, 0.6602564102564102, 0.41420562452977056, 0.7125, 0.42500000000000004, 0.605328, 0.8332498764052269), (0.6446666666666667, 0.515, 0.904, 0.6589891234804862, 0.4079200312827111, 0.7095, 0.41900000000000004, 0.611674, 0.8355329088221015), (0.6566666666666666, 0.522, 0.926, 0.6696600384862091, 0.4367795540086616, 0.724, 0.44799999999999995, 0.66555, 0.8551704188804932), (0.6566666666666666, 0.519, 0.932, 0.6683837733419189, 0.440680657661968, 0.7255, 0.45100000000000007, 0.6649079999999999, 0.854878764957969), (0.648, 0.5

[ 50   0   0 100]
 
Training/ Validation / Test BACC : 1.0 / 1.0 / 1.0  
Training/ Validation / Test Youden-index: 1.0 / 1.0 / 1.0  
Training/ Validation / Test MCC: 1.0 / 1.0 / 1.0  
Training/ Validation / Test AUC: 1.0 / 1.0 / 1.0  
Training/ Validation / Test AUPR: 1.0 / 1.0 / 1.0
[(0.8053333333333333, 0.813, 0.79, 0.8477580813347236, 0.5833369430134641, 0.8015, 0.603, 0.892695, 0.9441783122410885), (0.632, 0.667, 0.562, 0.7073170731707317, 0.2195429209159519, 0.6145, 0.2290000000000001, 0.70579, 0.868510348191698), (0.5766666666666667, 0.541, 0.648, 0.6301688992428655, 0.1783636485172738, 0.5945, 0.18900000000000006, 0.6118030000000001, 0.833540353777167), (0.57, 0.569, 0.572, 0.638250140213124, 0.13306494413193443, 0.5705, 0.14100000000000001, 0.631909, 0.8396869130525373), (0.63, 0.733, 0.424, 0.7253834735279565, 0.15874645173742327, 0.5785, 0.15700000000000003, 0.7106859999999999, 0.8663547086220278)]
[ 50   0   0 100]
 
Training/ Validation / Test BACC : 1.0 / 1.0 / 1.0  
Train

...

[(0.5953333333333334, 0.571, 0.644, 0.6529445397369925, 0.20270412412166736, 0.6074999999999999, 0.21499999999999986, 0.61134, 0.8419645531374833), (0.6013333333333334, 0.572, 0.66, 0.6567164179104478, 0.21874414211222037, 0.616, 0.23199999999999998, 0.615202, 0.8432727933980803), (0.5926666666666667, 0.569, 0.64, 0.6506575185820469, 0.19704726484385338, 0.6045, 0.20900000000000007, 0.605596, 0.8401009599776758), (0.592, 0.569, 0.638, 0.6502857142857142, 0.1951614716074871, 0.6034999999999999, 0.20699999999999985, 0.6041719999999999, 0.8396178445814709), (0.592, 0.57, 0.636, 0.6506849315068494, 0.19421935312483274, 0.603, 0.20599999999999996, 0.609208, 0.8412432346859661), (0.594, 0.571, 0.64, 0.6521987435751, 0.19893288460312472, 0.6054999999999999, 0.21099999999999985, 0.609928, 0.8415161013578165), (0.5973333333333334, 0.57, 0.652, 0.6536697247706422, 0.20931030526815694, 0.611, 0.22199999999999998, 0.6100859999999999, 0.8415576318740446), (0.5893333333333334, 0.569, 0.63, 0.6488027

[ 50   0   0 100]
 
Training/ Validation / Test BACC : 1.0 / 1.0 / 1.0  
Training/ Validation / Test Youden-index: 1.0 / 1.0 / 1.0  
Training/ Validation / Test MCC: 1.0 / 1.0 / 1.0  
Training/ Validation / Test AUC: 1.0 / 1.0 / 1.0  
Training/ Validation / Test AUPR: 1.0 / 1.0 / 1.0
[(0.8493333333333334, 0.811, 0.926, 0.8777056277056277, 0.7008591797395617, 0.8685, 0.7370000000000001, 0.9416839999999999, 0.9712549427291648), (0.854, 0.834, 0.894, 0.8839427662957074, 0.6981107639469836, 0.864, 0.728, 0.9301609999999999, 0.9667795202716998), (0.836, 0.802, 0.904, 0.867027027027027, 0.6716199097461456, 0.853, 0.706, 0.927513, 0.9649882862807306), (0.836, 0.834, 0.84, 0.8714733542319749, 0.6512128967598624, 0.837, 0.6739999999999999, 0.904044, 0.944922237993306), (0.8686666666666667, 0.843, 0.92, 0.8953797132235793, 0.7309482090097352, 0.8815, 0.7629999999999999, 0.9465979999999999, 0.9658337224594302), (0.8573333333333333, 0.83, 0.912, 0.8858057630736392, 0.7093262376934859, 0.871, 0.742

[ 49   1   0 100]
 
Training/ Validation / Test BACC : 1.0 / 1.0 / 0.99  
Training/ Validation / Test Youden-index: 1.0 / 1.0 / 0.98  
Training/ Validation / Test MCC: 1.0 / 1.0 / 0.9850365626224087  
Training/ Validation / Test AUC: 1.0 / 1.0 / 1.0  
Training/ Validation / Test AUPR: 1.0 / 1.0 / 1.0
[(0.5453333333333333, 0.564, 0.508, 0.6232044198895028, 0.06810052246069989, 0.536, 0.07200000000000006, 0.5668960000000001, 0.8226362829042463), (0.554, 0.568, 0.526, 0.6293628808864266, 0.08886331560375209, 0.5469999999999999, 0.09399999999999986, 0.5801759999999999, 0.8309877065080107), (0.5446666666666666, 0.568, 0.498, 0.6245189664650908, 0.062490418171125585, 0.5329999999999999, 0.06599999999999984, 0.577088, 0.8279890219207561), (0.5946666666666667, 0.569, 0.646, 0.6517754868270332, 0.202706826902183, 0.6074999999999999, 0.21499999999999986, 0.608332, 0.8408546020260951), (0.5613333333333334, 0.565, 0.554, 0.6319910514541387, 0.11233856139153842, 0.5595, 0.119, 0.581084, 0.829015069

[ 50   0   0 100]
 
Training/ Validation / Test BACC : 0.9975 / 0.99 / 1.0  
Training/ Validation / Test Youden-index: 0.9950000000000001 / 0.98 / 1.0  
Training/ Validation / Test MCC: 0.9962523291023692 / 0.9850365626224087 / 1.0  
Training/ Validation / Test AUC: 0.999921875 / 1.0 / 1.0  
Training/ Validation / Test AUPR: 0.9999604166157916 / 1.0 / 1.0
[(0.6666666666666666, 0.59, 0.82, 0.7023809523809523, 0.3882464347777672, 0.705, 0.4099999999999999, 0.6964359999999999, 0.8663139397249323), (0.658, 0.591, 0.792, 0.6973451327433628, 0.36207074336422396, 0.6915, 0.383, 0.68513, 0.8629207147293332), (0.664, 0.589, 0.814, 0.7003567181926278, 0.3815234258497175, 0.7015, 0.403, 0.6940059999999999, 0.8673491235917257), (0.668, 0.591, 0.822, 0.7035714285714286, 0.39108726234931185, 0.7064999999999999, 0.4129999999999998, 0.7001679999999999, 0.8697173208075358), (0.658, 0.59, 0.794, 0.6969875959834613, 0.3630887909636332, 0.692, 0.3839999999999999, 0.693838, 0.8657689960217985), (0.66, 0.58

[ 49   1   0 100]
 
Training/ Validation / Test BACC : 1.0 / 1.0 / 0.99  
Training/ Validation / Test Youden-index: 1.0 / 1.0 / 0.98  
Training/ Validation / Test MCC: 1.0 / 1.0 / 0.9850365626224087  
Training/ Validation / Test AUC: 1.0 / 1.0 / 1.0  
Training/ Validation / Test AUPR: 1.0 / 1.0 / 1.0
[(0.71, 0.585, 0.96, 0.7289719626168224, 0.5237117278309473, 0.7725, 0.5449999999999999, 0.75309, 0.8932822428695488), (0.736, 0.613, 0.982, 0.7558569667077681, 0.5693240335657713, 0.7975, 0.595, 0.848086, 0.9321426036823675), (0.7033333333333334, 0.58, 0.95, 0.7227414330218068, 0.5092976435787194, 0.7649999999999999, 0.5299999999999998, 0.6986319999999999, 0.8729682855737939), (0.7053333333333334, 0.58, 0.956, 0.7240948813982522, 0.5154818493121818, 0.768, 0.536, 0.7074900000000001, 0.876596391055832)]
[ 50   0   0 100]
 
Training/ Validation / Test BACC : 1.0 / 1.0 / 1.0  
Training/ Validation / Test Youden-index: 1.0 / 1.0 / 1.0  
Training/ Validation / Test MCC: 1.0 / 1.0 / 1.0  
Train

In [32]:
perf_bc_bn_r6g_1.groupby('Model').mean()

,Accuracy,Sensitivity,Specificity,F1,MCC,BACC,Yoden Index,AUROC,AUPR
Model,,,,,,,,,
DT,0.533600,0.7717,0.0574,0.670132,-0.208691,0.41455,-0.1709,0.414550,0.763080
LR,0.647467,0.5182,0.9060,0.662191,0.412919,0.71210,0.4242,0.628435,0.841664
LinSVM,0.644067,0.5019,0.9284,0.653506,0.426235,0.71515,0.4303,0.564325,0.819571
MLP,0.671333,0.9903,0.0334,0.800587,0.099404,0.51185,0.0237,0.863290,0.938271
NB,0.707600,0.6035,0.9158,0.733434,0.494526,0.75965,0.5193,0.801033,0.910350
RBFSVM,0.698400,0.9920,0.1112,0.814317,0.241535,0.55160,0.1032,0.927774,0.967732
RF,0.630933,0.6795,0.5338,0.706139,0.214081,0.60665,0.2133,0.705941,0.866847


In [33]:
perf_bc_bn_r6g_2.groupby('Model').mean()

,Accuracy,Sensitivity,Specificity,F1,MCC,BACC,Yoden Index,AUROC,AUPR
Model,,,,,,,,,
DT,0.758333,0.7749,0.7252,0.809346,0.485678,0.75005,0.5001,0.750050,0.887350
LR,0.593133,0.5698,0.6398,0.651244,0.197617,0.60480,0.2096,0.607413,0.840669
LinSVM,0.558600,0.5664,0.5430,0.631244,0.103309,0.55470,0.1094,0.582250,0.829972
MLP,0.709467,0.5840,0.9604,0.728241,0.523318,0.77220,0.5444,0.727506,0.883887
NB,0.577733,0.7091,0.3150,0.691245,0.024783,0.51205,0.0241,0.508321,0.787288
RBFSVM,0.665667,0.5893,0.8184,0.701519,0.386102,0.70385,0.4077,0.699216,0.869206
RF,0.840800,0.8142,0.8940,0.872019,0.676343,0.85410,0.7082,0.927493,0.963191


In [34]:
pd.concat([perf_bc_bn_r6g_1, perf_bc_bn_r6g_2], axis=0).groupby('Model').mean()

,Accuracy,Sensitivity,Specificity,F1,MCC,BACC,Yoden Index,AUROC,AUPR
Model,,,,,,,,,
DT,0.645967,0.77330,0.3913,0.739739,0.138494,0.582300,0.16460,0.582300,0.825215
LR,0.620300,0.54400,0.7729,0.656718,0.305268,0.658450,0.31690,0.617924,0.841166
LinSVM,0.601333,0.53415,0.7357,0.642375,0.264772,0.634925,0.26985,0.573287,0.824771
MLP,0.690400,0.78715,0.4969,0.764414,0.311361,0.642025,0.28405,0.795398,0.911079
NB,0.642667,0.65630,0.6154,0.712339,0.259655,0.635850,0.27170,0.654677,0.848819
RBFSVM,0.682033,0.79065,0.4648,0.757918,0.313819,0.627725,0.25545,0.813495,0.918469
RF,0.735867,0.74685,0.7139,0.789079,0.445212,0.730375,0.46075,0.816717,0.915019
